In [ ]:
#import arcpy
from arcpy import env
from collections import defaultdict
from  statistics import mean
import numpy
import time
start_time = time.time()

def createFishnet ():
    
    # set workspace environment
    # arcpy.env.workspace = r"C:\Users\ditsuhi\Desktop\arcMap\madrid\traf2020dec"
    #env.workspace = r"C:\Users\ditsuhi\Desktop\arcMap\madrid"
    # Set coordinate system of the output fishnet
    #env.outputCoordinateSystem = arcpy.SpatialReference("ETRS_1989_UTM_Zone_30N")
    if arcpy.Exists(r"C:\Users\ditsuhi\Desktop\arcMap\madrid\traf2020dec\fishnetAirMetTraf.shp"):
        arcpy.Delete_management(r"C:\Users\ditsuhi\Desktop\arcMap\madrid\traf2020dec\fishnetAirMetTraf.shp")

    outFeatureClass = "fishnetAirMetTraf.shp"
    # Set the origin of the fishnet
    originCoordinate = '434215.234430378 4466449.72526323'
    # Set the orientation
    yAxisCoordinate = '434215.234430378 4466459.72526323'
    # Enter 0 for width and height - these values will be calcualted by the tool
    cellSizeWidth = '1000'
    cellSizeHeight = '1000'
    # Number of rows and columns together with origin and opposite corner 
    # determine the size of each cell 
    numRows =  '0'
    numColumns = '0'
    oppositeCoorner = '451215.234430378 4486449.72526323'
    # Create a point label feature class 
    labels = 'NO_LABELS'
    # Extent is set by origin and opposite corner - no need to use a template fc
    templateExtent = r'C:\Users\ditsuhi\Desktop\arcMap\madrid\fishnetMeteorStation.shp'
    # Each output cell will be a polygon
    geometryType = 'POLYGON'
    arcpy.CreateFishnet_management(outFeatureClass, originCoordinate, yAxisCoordinate, cellSizeWidth, cellSizeHeight, numRows, numColumns, oppositeCoorner, labels, templateExtent, geometryType)    


    
def makeFeatureLayerAndTableViewUsingArcpy (currentDate, hour):
    
    
    FishnetAirMetTraf= r"C:\Users\ditsuhi\Desktop\arcMap\madrid\traf2020dec\fishnetAirMetTraf.shp"
    TrafficStation = r"C:\Users\ditsuhi\Desktop\arcMap\madrid\trafjune2019.shp"
    globals()[f"Traffic2019jun{currentDate.day}_{hour}"] = fr"C:\Users\ditsuhi\Desktop\arcMap\madrid\traf2020dec\2020dec31\2020dec31.gdb\sort_traf2019jun{currentDate.day}_{hour}_CopyRows"    
    arcpy.management.MakeFeatureLayer(FishnetAirMetTraf, "FishnetAirMetTraf")    
    arcpy.management.MakeFeatureLayer(TrafficStation, "TrafficStation")
    arcpy.MakeTableView_management(fr"C:\Users\ditsuhi\Desktop\arcMap\madrid\traf2020dec\2020dec31\2020dec31.gdb\sort_traf2019jun{currentDate.day}_{hour}_CopyRows", f"Traffic2019jun{currentDate.day}_{hour}")
    
    return FishnetAirMetTraf, TrafficStation, f"Traffic2019jun{currentDate.day}_{hour}"
        
def addFieldsToFishnet (trafColumn):
    for col in trafColumn:
        arcpy.management.AddField(FishnetAirMetTraf, col, "Double")
        
    
def spatialJoin (FishnetAirMetTraf, TrafficStation):
    outSpatialTrafficNew = r"C:\Users\ditsuhi\Desktop\arcMap\madrid\spatialTargetTrafficdata"
    arcpy.analysis.SpatialJoin(FishnetAirMetTraf, TrafficStation, outSpatialTrafficNew, "JOIN_ONE_TO_MANY", "KEEP_ALL")
    outSpatialTargetTrafficNew = r"C:\Users\ditsuhi\Desktop\arcMap\madrid\spatialTargetTrafficdata.shp"
    arcpy.management.MakeFeatureLayer(outSpatialTargetTrafficNew, "outSpatialTargetTrafficNew")
    station_FID_Code_dict_Traffic = defaultdict(list)
    with arcpy.da.SearchCursor(outSpatialTargetTrafficNew, ["Join_Count", "Target_FID", "id_1"]) as outSpatial_cursor:
        {station_FID_Code_dict_Traffic[row[1]].append(row[2])  for row in outSpatial_cursor  if row[0] == 1}
    
    #return station_FID_Code_dict, station_FID_Code_dict_Met, station_FID_Code_dict_Traffic    
    return station_FID_Code_dict_Traffic

                               
    
def trafDatameanById (FishnetAirMetTraf, station_FID_Code_dict_Traffic, trafCol, currentDate, hour):
    with arcpy.da.UpdateCursor(FishnetAirMetTraf, ["FID", trafCol]) as fishLabel_cursor:
        for x in fishLabel_cursor:
            for key, value in station_FID_Code_dict_Traffic.items():
                if x[0] == key:                    
                    with arcpy.da.SearchCursor(f"Traffic2019jun{currentDate.day}_{hour}", ["Fecha", "id", trafCol]) as trafdata_cursor:                    
                        intensList = [fecha[2] for fecha in trafdata_cursor  if fecha[1] in value and fecha[2] is not None]
                        if len(intensList) > 0:
                            x[1] = round(mean(intensList), 2)
                            fishLabel_cursor.updateRow(x)

    

def exportAttributeTableToCSVFile(cols, num, currentDate):
    nparr = arcpy.da.FeatureClassToNumPyArray(FishnetAirMetTraf, cols)
    numpy.savetxt(fr'C:\Users\ditsuhi\Desktop\arcMap\trafficCSVFiles\trafJun2019\fishnetAirMet{currentDate.year}_{currentDate.strftime("%b").lower()}_{currentDate.day}_{num}.csv', nparr, delimiter=",", fmt="%s", header = "FID, intensidad, ocupacion, carga, vmed")
    
    

arcpy.env.workspace = r"C:\Users\ditsuhi\Desktop\arcMap\madrid\traf2020dec"  
metMagnitude = {"80": "UV", "81": "windSpeed", "82": "windDir", "83": "Temp", "86": "Humidity", "87": "Pressure", "88": "SolarRad", "89": "Prec"}
trafColumn = ["intensidad", "ocupacion", "carga", "vmed"]
#allColumns = ["FID", "NO2", "UV", "windSpeed", "windDir", "Temp", "Humidity", "Pressure", "SolarRad", "Prec", "intensidad", "ocupacion", "carga", "vmed"]
allColumns = ["FID", "intensidad", "ocupacion", "carga", "vmed"]


from datetime import timedelta, date


for date_dec in range(4, 5):
    for hour in range(21, 24):
        arcpy.CopyRows_management(fr"C:\Users\ditsuhi\Desktop\arcMap\madrid\traf2020dec\content\trafJun2019\sort_traf2019jun{date_dec}_{hour}.csv", fr"C:\Users\ditsuhi\Desktop\arcMap\madrid\traf2020dec\2020dec31\2020dec31.gdb\sort_traf2019jun{date_dec}_{hour}_CopyRows")
        end_date = date(2019, 6, date_dec)               
        print(str(end_date))         

        createFishnet ()
        #FishnetAirMetTraf, TrafficStation, metStation, metdata, airQualityPoint, no2data = makeFeatureLayerAndTableViewUsingArcpy(itemNum)
        FishnetAirMetTraf, TrafficStation, globals()[f"Traffic2019jun{end_date.day}_{hour}"] = makeFeatureLayerAndTableViewUsingArcpy(end_date, hour)
        station_FID_Code_dict_Traffic = spatialJoin (FishnetAirMetTraf, TrafficStation)
        #station_FID_Code_dict, station_FID_Code_dict_Met, station_FID_Code_dict_Traffic = spatialJoin (FishnetAirMetTraf, TrafficStation, metStation, metdata, airQualityPoint)

        #addFieldsToFishnet (metMagnitude, trafColumn)
        addFieldsToFishnet (trafColumn)
        #airDataMatchedByStation (FishnetAirMetTraf, station_FID_Code_dict, itemNum, no2data, str(end_date))
        #metDataMatchedByStation (metMagnitude, FishnetAirMetTraf, station_FID_Code_dict_Met, metdata, itemNum, str(end_date))   
        for trafCol in trafColumn:           
            trafDatameanById(FishnetAirMetTraf, station_FID_Code_dict_Traffic, trafCol, end_date, hour)
        exportAttributeTableToCSVFile(allColumns, hour, end_date)

        arcpy.Delete_management(fr"C:\Users\ditsuhi\Desktop\arcMap\madrid\traf2020dec\2020dec31\2020dec31.gdb\sort_traf2019jun{date_dec}_{hour}_CopyRows")

        
        
print(f"{(time.time() - start_time)} second")  